# Test ELT-Harmoni simulator class

In [ ]:
import os
import hcipy
import matplotlib.pyplot as plt
import numpy as np

from pastis.config import CONFIG_PASTIS
from pastis.simulators.elt_imaging import ELTHarmoniSPC
import pastis.util

In [ ]:
sampling = CONFIG_PASTIS.getfloat('ELT', 'sampling')
optics_input = os.path.join(pastis.util.find_repo_location(), CONFIG_PASTIS.get('ELT', 'optics_path_in_repo'))
design = 'HSP1'
fpm_px = CONFIG_PASTIS.getint('ELT', 'fpm_px')
wvln = CONFIG_PASTIS.getfloat('ELT', 'lambda') * 1e-9

print(wvln)
print(type(wvln))

elt = ELTHarmoniSPC(input_dir=optics_input, sampling=sampling, wvln=wvln, spc_design=design, fpm_rad=fpm_px)

In [ ]:
elt.seg_pos.size

In [ ]:
elt_img, elt_direct, inter = elt.calc_psf(display_intermediate=True, ref=True, return_intermediate='efield')

In [ ]:
im = inter['before_fpm']
im_int = im.intensity / im.intensity.max()
#hcipy.imshow_field(np.log10(im_int) + elt.dh_mask*4, cmap='inferno', origin='lower', vmin=-7, vmax=-0)
hcipy.imshow_field(np.log10(im_int), cmap='inferno', origin='lower', vmin=-7, vmax=-0)
plt.colorbar()

In [ ]:
hcipy.imshow_field(np.log10(im_int), mask=elt.dh_mask, cmap='inferno', vmin=-10)
plt.colorbar()

In [ ]:
print(f'contrast: {np.mean(np.log10(im_int)[np.where(elt.dh_mask)])}')

## Segmented mirror

In [ ]:
#elt.create_segmented_mirror(2)  # TOOK EXTREMELY LONG TO RUN!! I let it run over night...

In [ ]:
aber_segnum = 798
seglist = np.random.randint(1, elt.seg_pos.size, size=aber_segnum)
#print(seglist)

In [ ]:
piston = np.random.uniform(low=1e-10, high=5e-10, size=(aber_segnum,))
tip = np.random.uniform(low=1e-10, high=5e-10, size=(aber_segnum,))
tilt = np.random.uniform(low=1e-10, high=5e-10, size=(aber_segnum,))

In [ ]:
for i in range(seglist.shape[0]):
    #elt.set_sm_segment(segid, zernike_number, amplitude)  # m of rms surface
    elt.set_sm_segment(i+1, 1, tip[i]*100000)

In [ ]:
elt.flatten()

In [ ]:
phase = inter['seg_mirror'].phase
hcipy.imshow_field(phase, cmap='Reds', origin='lower', mask=elt.aperture)
plt.colorbar()

In [ ]:
apod = inter['apod'].intensity
hcipy.imshow_field(apod, cmap='Greys_r', origin='lower', mask=elt.aperture)
plt.colorbar()